In [1]:
#Instantiate SparkSession
from pyspark.sql import SparkSession
spark=SparkSession \
.builder \
.appName('Predicting the Price of an automobile given a set of features ') \
.getOrCreate()

In [2]:
spark

In [3]:
# wget https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
# gsutil cp imports-85.data gs://dexdebra-123/datasets
# Use sparkSession to read csv file			
rawData=spark.read \
.format('csv') \
.option('header','false') \
.load('gs://dexdebra-123/datasets/imports-85.data')

In [4]:
rawData.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,...,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.80,...,109,mpfi,3.19,3.40,10.00,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.40,...,136,mpfi,3.19,3.40,8.00,115,5500,18,22,17450


In [47]:
rawData.describe()

DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string]

In [48]:
#Assign meaningful Headers
rawData=rawData.toDF(
    "symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors","body-style","drive-wheels",
          "engine-location","wheel-base","length","width","height","curb-weight","engine-type","num-of-cylinders",
          "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower","peak-rpm","city-mpg",
          "highway-mpg","price")

In [49]:
rawData.toPandas()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,...,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.80,...,109,mpfi,3.19,3.40,10.00,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.40,...,136,mpfi,3.19,3.40,8.00,115,5500,18,22,17450
5,2,?,audi,gas,std,two,sedan,fwd,front,99.80,...,136,mpfi,3.19,3.40,8.50,110,5500,19,25,15250
6,1,158,audi,gas,std,four,sedan,fwd,front,105.80,...,136,mpfi,3.19,3.40,8.50,110,5500,19,25,17710
7,1,?,audi,gas,std,four,wagon,fwd,front,105.80,...,136,mpfi,3.19,3.40,8.50,110,5500,19,25,18920
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.80,...,131,mpfi,3.13,3.40,8.30,140,5500,17,20,23875
9,0,?,audi,gas,turbo,two,hatchback,4wd,front,99.50,...,131,mpfi,3.13,3.40,7.00,160,5500,16,22,?


In [50]:
# Convert Numerical Values Read as String values to floating format

from pyspark.sql.functions import col

dataSet=rawData.select(
						col('price').cast('float'),
						col('make'),
						col('num-of-doors'),
						col('body-style'),
						col('drive-wheels'),
						col('wheel-base').cast('float'),
						col('curb-weight').cast('float'),
						col('num-of-cylinders'),
						col('engine-size').cast('float'),
						col('horsepower').cast('float'),
						col('peak-rpm').cast('float'),
)

In [51]:
dataSet

DataFrame[price: float, make: string, num-of-doors: string, body-style: string, drive-wheels: string, wheel-base: float, curb-weight: float, num-of-cylinders: string, engine-size: float, horsepower: float, peak-rpm: float]

In [52]:
dataSet.toPandas()

,price,make,num-of-doors,body-style,drive-wheels,wheel-base,curb-weight,num-of-cylinders,engine-size,horsepower,peak-rpm
0,13495.0,alfa-romero,two,convertible,rwd,88.599998,2548.0,four,130.0,111.0,5000.0
1,16500.0,alfa-romero,two,convertible,rwd,88.599998,2548.0,four,130.0,111.0,5000.0
2,16500.0,alfa-romero,two,hatchback,rwd,94.500000,2823.0,six,152.0,154.0,5000.0
3,13950.0,audi,four,sedan,fwd,99.800003,2337.0,four,109.0,102.0,5500.0
4,17450.0,audi,four,sedan,4wd,99.400002,2824.0,five,136.0,115.0,5500.0
5,15250.0,audi,two,sedan,fwd,99.800003,2507.0,five,136.0,110.0,5500.0
6,17710.0,audi,four,sedan,fwd,105.800003,2844.0,five,136.0,110.0,5500.0
7,18920.0,audi,four,wagon,fwd,105.800003,2954.0,five,136.0,110.0,5500.0
8,23875.0,audi,four,sedan,fwd,105.800003,3086.0,five,131.0,140.0,5500.0
9,NaN,audi,two,hatchback,4wd,99.500000,3053.0,five,131.0,160.0,5500.0


In [53]:
# Replace All '?' value in all cells with specific value('None') in dataFrame
print('Count Before: ' , dataSet.count())
dataSet=dataSet.replace('?',None)
dataSet=dataSet.dropna(how='any')
print('Count After: ' , dataSet.count())

('Count Before: ', 205)
('Count After: ', 197)


In [54]:
# Split the data into 80% for trainingData and 20% for testData
(trainingData,testData)=dataSet.randomSplit([0.8,0.2])

In [55]:
# Make the list of All Categorical Features in our dataset
# Convert them to one-hot-notation
categoricalFeatures= [
        'make',
        'num-of-doors',
        'body-style',
        'drive-wheels',
        'num-of-cylinders'
]


In [56]:
# Make a list of transformers we need

# Create StringIndexer for CategoricalFeatures using python list comprehension
from pyspark.ml.feature import StringIndexer

# Convert Numerical form of categories to One-hot-encoding using python list comprehension
from pyspark.ml.feature import OneHotEncoder

# InputFeatures are fed in the form of a DenseVector
# Use VectorAssember to Vectorize the features to a column called 'Features'
# Pass the columns we want to be vectorized and
# Output will be stored in a column called 'Features'
from pyspark.ml.feature import VectorAssembler

In [57]:
# Convert CategoricalFeatures to numeric form

indexers=[ StringIndexer(
inputCol=column,
outputCol=column + '_index',
handleInvalid='keep') for column in categoricalFeatures]

In [58]:
# Convert Numerical form of categories to One-hot-encoding using python list comprehension
from pyspark.ml.feature import OneHotEncoder
encoders=[ OneHotEncoder ( 
				inputCol=column + '_index',
				outputCol=column + '_encoded') for column  in categoricalFeatures]

In [59]:
# Make a list of requiredFeatures

requiredFeatures = [
         'make_encoded',
        'num-of-doors_encoded',
        'body-style_encoded',
        'drive-wheels_encoded',
        'wheel-base',
        'curb-weight',
        'num-of-cylinders_encoded',
        'engine-size',
        'horsepower',
        'peak-rpm'
]

# InputFeatures are fed in the form of a DenseVector
# Use VectorAssember to Vectorize the features to a column called 'Features'
# Pass the columns we want to be vectorized and
# Output will be stored in a column called 'Features'

assembler = VectorAssembler( inputCols=requiredFeatures , outputCol='Features')

In [60]:
# Instantiate LinearRegression Model
from pyspark.ml.regression import LinearRegression

lr=LinearRegression (
#Number of epochs for which we run training process. 
#An epoch is one run thru' entire training data set
maxIter=100, 
#Regularisation Parameter Alpha, Penalty that is applied to our model co-efficients
regParam=1.0,
#Similar to Alpha, except this is use to tune the model to move between Lasso and Ridge Regression
# 0.0 for Lasso Model
# 1.0 for Ridge Regression Model
elasticNetParam=0.8,
labelCol='price',
featuresCol='Features'
)


In [61]:
# Instantiate the Pipeline with various stages , indexers,encoders,assemblers and then our LinearRegression Estimator

from pyspark.ml import Pipeline
pipeline=Pipeline( stages=indexers + encoders + [assembler,lr] )

In [62]:
#Kickstart the Training Pipeline by calling fit() on trainingData
model=pipeline.fit(trainingData)

# Returned model is actually a pipeline model, it is not really a Linear Regression Model
# it contains all the stages of the training pipeline model


In [63]:
# To Access only LR Model, we can access last stage of the training pipeline
lrModel=model.stages[-1]

In [64]:
# We can 2 metrics to evaluate the LR Model

# R2 Score: Measure of how well the LR Model captures the variance in underlying training data.
print('Training R2 Score = ', lrModel.summary.r2)

# Higher the R2 ,better  the model

# RMSE : Gives average value by which our predictions vary from the actual price
print('Training RMSE = ', lrModel.summary.rootMeanSquaredError)

('Training R2 Score = ', 0.9477304331478545)
('Training RMSE = ', 1929.1595292305578)


In [65]:
# Get the number of features that went into the model using numFeatures 
lrModel.numFeatures
# one-hot-encoders have expanded the number of features in our training data

41

In [66]:

# Get the coefficients member variable on the model in DenseVector form
lrModel.coefficients

DenseVector([-2135.1013, -402.0468, -2310.0092, -1966.0602, -1286.4757, -625.9697, -477.9133, 1228.0989, -2427.2471, 8000.6201, -1928.9098, -1423.498, 900.1244, 8663.6238, 3758.1839, 11355.4584, 5810.2141, 41.5082, -1243.1891, -1239.7956, 159.2402, -159.2402, 9.3409, 5.3685, -861.2516, 1102.4337, 3627.4754, -50.1582, -4.2275, 213.0116, 120.7578, 3.6273, -1035.9663, 553.2064, -1258.361, 1825.7536, 3627.2633, -379.6604, 25.1309, 40.2175, -0.1033])

In [67]:
#length of coefficients corresponds to our features
len(lrModel.coefficients)

41

In [68]:
# Use this LR model to perform predictions on our testData set

predictions=model.transform(testData)
predictionsDF=predictions.toPandas()
predictionsDF.head()

# Output Result includes 'prediction' column


,price,make,num-of-doors,body-style,drive-wheels,wheel-base,curb-weight,num-of-cylinders,engine-size,horsepower,...,body-style_index,drive-wheels_index,num-of-cylinders_index,make_encoded,num-of-doors_encoded,body-style_encoded,drive-wheels_encoded,num-of-cylinders_encoded,Features,prediction
0,5151.0,chevrolet,two,hatchback,fwd,88.400002,1488.0,three,61.0,48.0,...,1.0,0.0,6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3449.997725
1,5389.0,mitsubishi,two,hatchback,fwd,93.699997,1918.0,four,92.0,68.0,...,1.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",4971.822519
2,6295.0,chevrolet,two,hatchback,fwd,94.500000,1874.0,four,90.0,70.0,...,1.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6133.388361
3,6338.0,toyota,two,hatchback,fwd,95.699997,2040.0,four,92.0,62.0,...,1.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5779.021690
4,6529.0,honda,two,hatchback,fwd,93.699997,1940.0,four,92.0,76.0,...,1.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",7122.990582


In [69]:
# Get the features and corresponding predictions for one instance in our testData set
predictionsDF.describe()
predictionsDF['Features'][0]

SparseVector(41, {18: 1.0, 21: 1.0, 23: 1.0, 27: 1.0, 30: 88.4, 31: 1488.0, 38: 61.0, 39: 48.0, 40: 5100.0})

In [70]:
# Use the Regression Evaluator that allows to measure how well our regression model performed on the test data set

from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator (
    labelCol='price', # Column that holds Actual Price
    predictionCol='prediction', # Column that holds the Predicted Price
    metricName='r2' # The Metric to use
)

In [71]:
r2=evaluator.evaluate(predictions)
print('Test R2 Score=',r2)

# little lower than our training data

('Test R2 Score=', 0.8853888684376925)


In [72]:
# Use the RegressionEvaluator to calculate RMSE on the testdata
evaluator=RegressionEvaluator (
labelCol='price', # Column that holds Actual Price
predictionCol='prediction', # Column that holds the Predicted Price
metricName='rmse' # The Metrics to use
)
rmse=evaluator.evaluate(predictions)
print('Test RMSE Score=',rmse)

# this is worse than our training data

('Test RMSE Score=', 1627.3136551638772)


In [73]:
predictionsPandaDF = predictions.select(
        col('price'),
        col('prediction')
).toPandas()

In [74]:
predictionsPandaDF.head()


,price,prediction
0,5151.0,3449.997725
1,5389.0,4971.822519
2,6295.0,6133.388361
3,6338.0,5779.021690
4,6529.0,7122.990582


In [75]:
# Plot the values to see how closely they tract one another

import matplotlib.pyplot as plt

plt.figure(figsize=(15,6))
plt.plot(predictionsPandaDF['price'], label='Actual')
plt.plot(predictionsPandaDF['prediction'], label='Predicted')
plt.ylabel('Price')
plt.legend()
plt.show()


# Prediction is orange, Actual is blue.
# they are pretty close

In [76]:
#ParamGridBuilder

  # Allows us to specify different values that we want to use
  # for the design parameters of our ML model

  #ParamGridBuilder will instantiate different models based on these design parameters
  #Run Training use our dataset and recommend the best model to us.


from pyspark.ml.tuning import ParamGridBuilder

paramGrid=ParamGridBuilder().addGrid(
    lr.maxIter,[10,50,100]).addGrid( # 10 or 50 or 100 epochs,which of these perform best.
    lr.regParam,[0.1,0.3,1.0]).addGrid( # 0.1,0.3 and 1.0
    lr.elasticNetParam,[0.0,0.8,1.0]).build() # Move from lasso regression to Pure Rigde Regression



In [77]:
# Use the RegressionEvaluator with RMSE metric to evaluate the model's that have been built using ParamGrid

evaluator=RegressionEvaluator (
labelCol='price',
predictionCol='prediction',
metricName='rmse'
)


In [78]:
# Use Cross Validation to help tune our model

# A Method to  train our model on a subset of data and test it using another subset that the model has not seen before

# The inputs are,
# (i)   Model Training Pipeline,
# (ii)  Params in our ParamGrid,
# (iii) Evaluator and
# (iv)  number of folds and that we want to specify in Cross Validation.

# The number of folds refers to number of subsets that we want to divide our data into.

from pyspark.ml.tuning import CrossValidator
crossval=CrossValidator (
estimator=pipeline, # Our Model Training Pipeline
estimatorParamMaps=paramGrid, # Params defined in ParamGrid
evaluator=evaluator, # Evaluator
numFolds=3 # Number of folds we want to specify for Cross Validation
)


In [79]:
# Start the Training Process
model=crossval.fit(trainingData)


In [80]:
# Once training is Complete,
# Access Linear Regression model from Pipeline
lrModel=model.bestModel.stages[-1]
lrModel



LinearRegression_4653a7ae3176098442f8

In [81]:
# Print the Parameters that produced the best model
print('maxIter=',lrModel._java_obj.getMaxIter())
print('elasticNetParam=',lrModel._java_obj.getElasticNetParam())
print('regParam=',lrModel._java_obj.getRegParam())

# Ridge Regression Model, 


('maxIter=', 10)
('elasticNetParam=', 1.0)
('regParam=', 0.3)


In [82]:
# Use the model for predictions

predictions=model.transform(testData)

# Use the RegressionEvaluator with R2 metric to evaluate the model
evaluator=RegressionEvaluator (
labelCol='price',
predictionCol='prediction',
metricName='r2'
)
r2=evaluator.evaluate(predictions)

print('Test R2 Score=%g' % r2)

# Much higher than R2 Score we saw earlier

Test R2 Score=0.869478


In [83]:
# Use the RegressionEvaluator with R2 metric to evaluate the model
evaluator=RegressionEvaluator (
labelCol='price',
predictionCol='prediction',
metricName='rmse'
)

rmse=evaluator.evaluate(predictions)
print('Test RMSE Score=%g' % rmse)



Test RMSE Score=1736.6


In [ ]:
    # Estimator and Transformers are chained to form ML Pipeline
    # Classifiers can be evaluated using Confusion Matrix
    # Other metrics to evaluate like precision and recall
    # Decision Tree and Random Forests Model for classification
